# Práctica Databricks Fundamentals  
## Unity Catalog · PySpark

En este notebook vamos a trabajar de forma práctica con Databricks, PySpark y Unity Catalog.

⚠️ **Importante**: sigue la guía HTML paso a paso.  
Aquí ejecutarás el código y completarás los ejercicios.


---
<h2>🧪 Flujo de la práctica</h2>
En esta práctica trabajaremos sobre un cluster ya levantado y un archivo CSV
disponible en el entorno.
<ol>
    <li>Cargar un dataset desde almacenamiento</li>
    <li>Explorar los datos con PySpark</li>
    <li>Aplicar transformaciones</li>
    <li>Persistir resultados como tabla Delta</li>
    <li>Inspeccionar el gobierno de datos con Unity Catalog</li>
</ol>

---

## Ejercicio 0 – Inicialización del entorno

Databricks ya proporciona una SparkSession activa.
Comprueba que el objeto `spark` está disponible.


In [0]:
spark


A continuación vamos a cargar las liberias que serán necesarias

In [0]:
# Imports básicos
from pyspark.sql.functions import col, sum as spark_sum


## 📥 Ejercicio 1 – Carga de datos

Vamos a cargar un archivo CSV con datos de ventas.
El archivo se encuentra en una ruta accesible desde el workspace.


In [0]:
# TODO:
# Carga el csv



Cuando lo tengas cargado, muestralo

In [0]:
# TODO:
# Muestra el df



Completa las siguientes tareas:

1. Muestra el esquema del DataFrame  
2. Cuenta el número total de registros  
3. Observa los tipos de datos inferidos por Spark


In [0]:
# TODO:
# 1. Mostrar el esquema
# 2. Contar el número de filas
# 3. Observar datos


## 🔧 Paso 2 – Transformaciones con PySpark

Vamos a crear una nueva columna calculada:

total_amount = quantity * price

Recuerda que las transformaciones en Spark son *lazy*.


In [0]:
# TODO:
# Escribe el codigo para crear la columna nueva

## ✍️ Ejercicio 2 – Transformaciones

Completa los siguientes pasos:

1. Filtra las ventas con `total_amount > 100`
2. Agrupa por `country`
3. Calcula el total de ventas por país


In [0]:
# TODO:
# - Filtrar total_amount > 100
# - Agrupar por country
# - Calcular la suma total de ventas


## 🧠 Paso 3 – Unity Catalog

Unity Catalog organiza los datos jerárquicamente:

Metastore  
└── Catalog  
    └── Schema  
        └── Table

Vamos a crear (si tenemos permisos) un catálogo y un esquema.


In [0]:
%sql
CREATE CATALOG IF NOT EXISTS ...;
USE CATALOG ...;

CREATE SCHEMA IF NOT EXISTS ...;
USE SCHEMA ...;


## 💾 Paso 4 – Crear una tabla Delta gestionada

Guardaremos el DataFrame transformado como una tabla gobernada
por Unity Catalog.


### ✍️ Ejercicio 3 - Gobierno del dato

In [0]:
# TODO:
# Ejecuta el guardado de la tabla
# Asegurate que la tabla se ha guardado correctamente con SQL y con PySpark
# Inspecciona los permisos de la tabla creada
# ¿Qué permisos necesita otro usuario para consultar esta tabla?


--------------------

## Soluciones


### Ejercicio 1

In [0]:
df = (
    spark.read
    .option("header", "true")
    .option("inferSchema", "true")
    .csv("/Volumes/master/repaso/datos/sales.csv")
)

display(df)
df.printSchema()
df.count()


### Ejercicio 2

In [0]:
df_transformed = (
    df
    .withColumn("total_amount", col("quantity") * col("price"))
)
display(df_transformed)

In [0]:
df_transformed_filtered = df_transformed.filter(col("total_amount") > 100)

df_agg = (
    df_transformed_filtered
    .groupBy("country")
    .agg(spark_sum("total_amount").alias("total_sales"))
)

display(df_agg)


### Ejercicio 3

In [0]:
df_transformed.write.mode("overwrite").saveAsTable("master.repaso.sales_table")

In [0]:
df_leido = spark.table("master.repaso.sales_table")
display(df_leido)

In [0]:
%sql
SELECT * FROM master.repaso.sales_table;
